## Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
import evaluate

In [ ]:
device = torch.device('cuda')

## Loading Dataset

In [ ]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [ ]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [ ]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [ ]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [ ]:
train_dataset = train_dataset.rename_columns({"text_content": "text", "status": "label"})
validation_dataset = validation_dataset.rename_columns({"text_content": "text", "status": "label"})
test_dataset = test_dataset.rename_columns({"text_content": "text", "status": "label"})

columns_to_remove = [
  'tags_onehot',
  'unrecognized_tags_count',
  'reputation',
  'undeleted_answers',
  'user_life_days',
  'title'
]

train_dataset = train_dataset.remove_columns(columns_to_remove)
validation_dataset = validation_dataset.remove_columns(columns_to_remove)
test_dataset = test_dataset.remove_columns(columns_to_remove)

### Tokenizing Data

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
MAX_TEXT_CONTENT = 128

def tokenize_func(batch):
  tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True, max_length=MAX_TEXT_CONTENT, return_tensors="pt")
  tokenized_batch["labels"] = [label2id[label] for label in batch["label"]]
  return tokenized_batch

In [ ]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [ ]:
from custom_model import AutoCompositeModel

model = AutoCompositeModel(device).to(device)

In [ ]:
model.my_new_layers[0].weight.dtype

In [ ]:
input = torch.tensor(tokenized_test_dataset['input_ids'][:10]).float().to(device)
input.dtype

## Training Setup

### Data Loaders

In [ ]:
from torch.utils.data import DataLoader
training_loader = DataLoader(tokenized_train_dataset, batch_size=1, shuffle=True)
validation_loader = DataLoader(tokenized_validation_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=True)

### Loss Function

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [ ]:
# Optimizers specified in the torch.optim package
optimizer = torch.optim.SGD(model.my_new_layers.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [ ]:
from training_own import Trainer, TrainerConfiguration

config = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)

In [ ]:
trainer = Trainer(model=model, trainer_configuration=config, input_column='input_ids', output_column='labels')

In [ ]:
trainer.train_one_epoch(logging_frequency=50)

In [ ]:
# for i, data in enumerate(training_loader):
#   if i > 0: break
#   # print(i, torch.FloatTensor(data['input_ids']))
  
#   # a = [torch.FloatTensor([1]).view(1, -1), torch.FloatTensor([2]).view(1, -1)]
#   stacked = torch.stack(data['input_ids']).float()
#   print(stacked.dtype)

#   # model(data['input_ids'])
  